In [ ]:
%matplotlib notebook

import functools

import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import xarray as xr

import lifetimes

In [ ]:
# Create fake dataset of two temporarily variable elliptical data regions on a grid
#dataset = lifetimes.testing.create_dummy_ecmwf_ifs_hres_dataset(
#    grid_size=(10, 10)
#)
#ds = dataset.as_xarray()

# Or load from local file
path = '/home/fabian/Documents/MAELSTROM/data/pca/ecmwf_ifs_hres_daily_temperature_averages_jan_dec_2020.nc'
ds = xr.open_dataset(path)

data = ds["t"]

# determing modes (perform spatio-temporal PCA)
modes = [lifetimes.modes.Modes(feature=data)]
pca_partial_method = functools.partial(
    lifetimes.modes.methods.spatio_temporal_principal_component_analysis,
    time_coordinate="time",
    latitude_coordinate="latitude",
)
[pca] = lifetimes.modes.determine_modes(modes=modes, method=pca_partial_method)

In [ ]:
anim = lifetimes.plotting.animate_timeseries(data)

In [ ]:
lifetimes.plotting.plot_scree_test(pca, variance_ratio=0.95)
n_components = 6

In [ ]:
lifetimes.plotting.plot_first_three_components_timeseries(pca)

In [ ]:
clusters = lifetimes.modes.methods.find_principal_component_clusters(
    pca, n_components=n_components, n_clusters=4,
)
lifetimes.plotting.plot_first_three_components_timeseries_clusters(clusters)

In [ ]:
clusters.labels.plot()

In [ ]:
data_weighted = lifetimes.utils.weight_by_latitudes(
    data, latitudes="latitude"
)
data_standardized = lifetimes.utils.standardize(data_weighted)

In [ ]:
anim = animate(data_standardized)
plt.plot()

In [ ]:
data_reshaped = lifetimes.utils.reshape_spatio_temporal_numpy_array(
    data_standardized.values
)

In [ ]:
d, theta = lifetimes.modes.methods.indicators(
    data_reshaped, 
    metric="euclidean", 
    q=0.97, 
    pareto_fit="scipy", 
    theta_fit="sueveges", 
)

indicators = xr.Dataset(
    data_vars={
        "d": (["time"], d),
        "theta": (["time"], theta),
    },
    coords={"time": data.coords["time"]},
)

In [ ]:
indicators["d"].plot()

In [ ]:
indicators["theta"].plot()